In [4]:
import pandas as pd
from pandas import Series, DataFrame
from pandas import read_html
import sqlalchemy 
from sqlalchemy import create_engine
import psycopg2
import currencyapicom


In [11]:
#importing dates from invoice table
host = 'localhost'
database = 'chinook'
user = 'postgres'
password = 'postgres'

connection = psycopg2.connect(host=host, database=database, user=user, password=password)
cursor = connection.cursor()
query = """
    SELECT DISTINCT DATE_TRUNC('day', d.invoicedate) AS invoice_date
    FROM stg.invoice d
    WHERE d.invoicedate >= '2022-11-01'::date
    ;
"""
cursor.execute(query)
dates_from_database = [date[0] for date in cursor.fetchall()]
cursor.close()
connection.close()


In [18]:
#importing ILS values and creating dataframe
api_key = 'cur_live_RcMFHRI0zjSZyTuoSGSBKStpw9G8eXwzLd1nzGSj'
client = currencyapicom.Client(api_key)
data_list = []
for date in dates_from_database:
    result = client.historical(date.strftime('%Y-%m-%d'), currencies=['USD', 'ILS'])

    usd_to_nis_rate = result.get('data', {}).get('ILS', {}).get('value', None)

    
    if usd_to_nis_rate is not None:
        
        data_list.append({'Date': date, 'Currency': 'ILS', 'Rate': usd_to_nis_rate})


df_results = pd.DataFrame(data_list)
print(df_results)

         Date Currency      Rate
0  2022-11-25      ILS  3.422285
1  2022-12-05      ILS  3.401444
2  2022-11-24      ILS  3.419938
3  2022-12-17      ILS  3.463707
4  2022-11-29      ILS  3.437551
5  2022-12-13      ILS  3.399205
6  2022-11-16      ILS  3.432195
7  2022-11-18      ILS  3.466650
8  2022-12-12      ILS  3.437854
9  2022-11-01      ILS  3.536732
10 2022-11-08      ILS  3.534891
11 2022-12-14      ILS  3.399255
12 2022-12-09      ILS  3.419105
13 2022-11-06      ILS  3.548679
14 2022-11-28      ILS  3.433037
15 2022-11-21      ILS  3.475548
16 2022-12-08      ILS  3.433645
17 2022-12-04      ILS  3.407240
18 2022-11-05      ILS  3.548694
19 2022-11-12      ILS  3.416301


In [19]:
# exporting the data to a table in chinook database
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
database = 'chinook'

connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'

engine = create_engine(connection_string)

schema_name = 'stg'
table_name = 'currency'

df_results.to_sql(table_name, engine, schema=schema_name, index=False, if_exists='replace')



20